In [38]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [39]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=154):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [40]:
# def get_naver_articles(urls):
#     articles, titles, dates, press = [], [], [], []
    
#     for url in urls:
#         html = requests.get(url, headers=headers)
#         soup = BeautifulSoup(html.text, "html.parser")

#         article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
#         article_titles = [title.text for title in soup.select(".news_tit")]
#         article_infos = [info.text for info in soup.select(".info")]

#         new_urls = []
#         idx = 0

#         while idx < len(article_links) - 1:
#             if 'news.naver.com' in article_links[idx]:
#                 # Check if the next link also contains 'news.naver.com'
#                 if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
#                     new_urls.append(article_links[idx + 1])
#                     idx += 2
#                 else:
#                     new_urls.append(article_links[idx])
#                     idx += 1
#             elif 'news.naver.com' in article_links[idx + 1]:
#                 new_urls.append(article_links[idx + 1])
#                 idx += 2
#             else:
#                 new_urls.append(article_links[idx])
#                 idx += 1

#         # 마지막 요소 처리
#         if idx == len(article_links) - 1:
#             new_urls.append(article_links[idx])

#         date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

#         current_dates = []
#         current_press = []

#         for idx, info in enumerate(article_infos):
#             match = date_pattern.search(info)
#             if match:
#                 current_dates.append(match.group(0))
                
#                 if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
#                     current_press.append(article_infos[idx - 1])

#         articles.extend(new_urls)
#         titles.extend(article_titles)
#         dates.extend(current_dates)
#         press.extend(current_press)

#         time.sleep(random.uniform(1, 1.5))
        
#     return articles, titles, dates, press

In [41]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles_temp = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        # 각 페이지에서 URL이 10개가 아닌 경우 해당 페이지의 URL 출력
        if len(new_urls) % 10 != 0:
            print(f"Warning: {url} contains {len(article_links)} links instead of 10.")
        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []
        current_titles = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press_name = article_infos[idx - 1]
                    
                    if "CHIEF EXECUTIVE" not in current_press_name:
                        current_press.append(current_press_name)
                        current_titles.append(article_titles_temp[len(current_dates)])
                        current_dates.append(match.group(0))

        articles.extend(new_urls[:len(current_titles)])  # articles length should match with titles
        titles.extend(current_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(random.uniform(1, 1.5))
        
    return articles, titles, dates, press

In [42]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time

In [43]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [44]:
search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")



100%|██████████| 1532/1532 [02:19<00:00, 10.96it/s]

Length of article_urls: 1532
Length of article_titles: 1532
Length of article_dates: 1532
Length of article_press: 1532
Length of contents: 1532
Length of article_time: 1532
            date                 time  \
0     2020.09.22                        
1     2020.09.22                        
2     2020.09.22                        
3     2020.09.22  2020-09-22 07:54:39   
4     2020.09.22                        
...          ...                  ...   
1526  2020.10.31                        
1527  2020.10.31                        
1528  2020.10.31  2020-10-31 10:17:00   
1529  2020.10.31                        
1530  2020.10.31                        

                                                  title  \
0                          [CEO칼럼] 스타워즈와 우주화폐, 그리고 블록체인   
1     [오알뉴 0921] 그 귀하던 마스크가…'개당 10센트' 폭락/ 주문 안한 물건 배...   
2         [아주경제 코이너스 브리핑] 김병욱 의원, 가상자산 업권법 제정 세미나 개최...   
3                         서울서 전국으로…방문판매업체·설명회발 '무차별 확산'   
4          코로나19 재확산 우려에 美증시·금값·비트코인 동

In [45]:
df

,date,time,title,content,press,link
0,2020.09.22,,"[CEO칼럼] 스타워즈와 우주화폐, 그리고 블록체인",,아주경제,http://www.ajunews.com
1,2020.09.22,,[오알뉴 0921] 그 귀하던 마스크가…'개당 10센트' 폭락/ 주문 안한 물건 배...,,미주중앙일보,http://koreadaily.com
2,2020.09.22,,"[아주경제 코이너스 브리핑] 김병욱 의원, 가상자산 업권법 제정 세미나 개최...",,아주경제,http://www.ajunews.com
3,2020.09.22,2020-09-22 07:54:39,서울서 전국으로…방문판매업체·설명회발 '무차별 확산',\n[앵커]그런데 서울 강남이 더 우려스러운 건 여기서 끝이 아니라는 겁니다. 이 ...,JTBC,https://n.news.naver.com/mnews/article/437/000...
4,2020.09.22,,"코로나19 재확산 우려에 美증시·금값·비트코인 동반 약세…""향후 변동성 커...",,코인리더스,http://coinreaders.com
...,...,...,...,...,...,...
1526,2020.10.31,,[인포그래픽 뉴스/바이비트] 숫자로 보는 오늘의 비트·알트코인 시세(10월31...,,코인리더스,http://coinreaders.com
1527,2020.10.31,,"KAIST, WEF와 협력해 블록체인 글로벌 표준 보고서 발간",,보안뉴스,http://www.boannews.com
1528,2020.10.31,2020-10-31 10:17:00,'로또 33억·비트코인 100억' 그들의 현재는?…'그것이 알고싶다' 열풍과 조작,\n\n\n\n\n '그것이 알고싶다' [SBS][아이뉴스24 정상호 기자] 올해 ...,아이뉴스24,https://n.news.naver.com/mnews/article/031/000...
1529,2020.10.31,,"비트코인, 장중 14,000달러 돌파…2018년 1월 이후 최고가 기록",,코인리더스,http://coinreaders.com


In [ ]:
# end_date = '2017.10.09'

In [12]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
df.to_csv(filename, encoding='utf-8-sig', index=False)

In [22]:
article_urls[390:400]

['https://n.news.naver.com/mnews/article/015/0004426402?sid=102',
 'http://www.thepublic.kr/',
 'http://www.dailysecu.com',
 'http://coinreaders.com',
 'https://n.news.naver.com/mnews/article/030/0002906028?sid=105',
 'http://www.epnc.co.kr',
 'http://www.thepublic.kr/',
 'http://www.ciokorea.com',
 'http://www.breaknews.com',
 'http://www.g-enews.com/']

In [ ]:
article_urls.insert(394, "https://www.chiefexe.com/news/ArticleView.asp?listId=Mjc0NHx8bGltaXRfZmFsc2Ug")

In [46]:
df1 = df

In [47]:
df1

,date,time,title,content,press,link
0,2020.09.22,,"[CEO칼럼] 스타워즈와 우주화폐, 그리고 블록체인",,아주경제,http://www.ajunews.com
1,2020.09.22,,[오알뉴 0921] 그 귀하던 마스크가…'개당 10센트' 폭락/ 주문 안한 물건 배...,,미주중앙일보,http://koreadaily.com
2,2020.09.22,,"[아주경제 코이너스 브리핑] 김병욱 의원, 가상자산 업권법 제정 세미나 개최...",,아주경제,http://www.ajunews.com
3,2020.09.22,2020-09-22 07:54:39,서울서 전국으로…방문판매업체·설명회발 '무차별 확산',\n[앵커]그런데 서울 강남이 더 우려스러운 건 여기서 끝이 아니라는 겁니다. 이 ...,JTBC,https://n.news.naver.com/mnews/article/437/000...
4,2020.09.22,,"코로나19 재확산 우려에 美증시·금값·비트코인 동반 약세…""향후 변동성 커...",,코인리더스,http://coinreaders.com
...,...,...,...,...,...,...
1526,2020.10.31,,[인포그래픽 뉴스/바이비트] 숫자로 보는 오늘의 비트·알트코인 시세(10월31...,,코인리더스,http://coinreaders.com
1527,2020.10.31,,"KAIST, WEF와 협력해 블록체인 글로벌 표준 보고서 발간",,보안뉴스,http://www.boannews.com
1528,2020.10.31,2020-10-31 10:17:00,'로또 33억·비트코인 100억' 그들의 현재는?…'그것이 알고싶다' 열풍과 조작,\n\n\n\n\n '그것이 알고싶다' [SBS][아이뉴스24 정상호 기자] 올해 ...,아이뉴스24,https://n.news.naver.com/mnews/article/031/000...
1529,2020.10.31,,"비트코인, 장중 14,000달러 돌파…2018년 1월 이후 최고가 기록",,코인리더스,http://coinreaders.com


In [18]:
len(df1)
len(df2)
len(df3)
print(len(df1)+len(df2)+len(df3))

NameError: name 'df2' is not defined

In [74]:
result

,date,time,title,content,press,link
0,2019.03.01,2019-03-01 00:01:01,"[가상화폐 뉴스] 03월 01일 00시 00분 비트코인(0.14%), 이오스(2.0...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
1,2019.03.01,,"암호화페(가상화폐)대표 종목 비트코인시황 상승세,3월 1일 00시 기준 '비트코...",,내외경제tv,http://www.nbntv.co.kr
2,2019.03.01,,비트코인 -0.02%↓ 이더리움 0.40%↑ 리플 0.58%↑ 퀀텀 -1.30%↓ ...,,로이슈,http://www.lawissue.co.kr
3,2019.03.01,,비트코인 0.07%↑ 이더리움 0.40%↑ 리플 0% 라이트코인 0.40%↑ 대시 ...,,로이슈,http://www.lawissue.co.kr
4,2019.03.01,,비트코인 0.09%↑ 이더리움 0.40%↑ 리플 0.58%↑ 대시 -0.04%↓ 에...,,로이슈,http://www.lawissue.co.kr
...,...,...,...,...,...,...
13034,2019.03.31,,[암호화폐뉴스] 31일 가상화폐 대표 종목 상승… 비트코인 23시 17분 기준 시...,,CCTV뉴스,http://www.cctvnews.co.kr
13035,2019.03.31,2019-03-31 23:31:00,"[가상화폐 뉴스] 03월 31일 23시 30분 비트코인(1.15%), 질리카(7.4...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
13036,2019.03.31,,[암호화폐 시황] 비트코인 0.06%↑ 비트코인캐시 0.53%↑ 비트코인골드 0% ...,,빅데이터뉴스,http://www.thebigdata.co.kr/
13037,2019.03.31,,"[세계 거래소별 가상화폐 시황] 31일, 23시 30분 현재",,로이슈,http://www.lawissue.co.kr


In [115]:
result = pd.concat([df1, df2, df4, df5], ignore_index=True)
result.drop_duplicates(keep='first', inplace=True, ignore_index=True)
result

,date,time,title,content,press,link
0,2020.05.01,,비트코인 -0.41%↓ 이더리움 -0.16%↓ 리플 -0.38%↓ 에이다 -0.33...,,로이슈,http://www.lawissue.co.kr
1,2020.05.01,,비트코인 -0.53%↓ 이더리움 -0.31%↓ 리플 -0.38%↓ 퀀텀 0.53%↑...,,로이슈,http://www.lawissue.co.kr
2,2020.05.01,,비트코인 -0.57%↓ 이더리움 -0.23%↓ 리플 -0.45%↓ 라이트코인 0.5...,,로이슈,http://www.lawissue.co.kr
3,2020.05.01,,"[코인빗 시황] 0시 1분 현재 비트코인 0.06% 오른 1068만원, 이더리움 0...",,로이슈,http://www.lawissue.co.kr
4,2020.05.01,,[레이븐랜드에 듣다]③ 디지털 자산 증권의 발행 플랫폼 레이븐코인 잠재력 무...,,블록체인밸리,http://www.fintechpost.co.kr
...,...,...,...,...,...,...
11113,2020.07.31,2020-07-31 17:34:05,디지털 화폐 찍는 중국…'위안화 굴기' 속도 낸다,\n'달러패권'에 도전장인민銀 '디지털위안' 사업 추진中 일상으로 빠르게 확산될 듯...,A22면 1단,https://n.news.naver.com/mnews/article/015/000...
11114,2020.07.31,,비트코인 실제 시장 지배력 80%가 넘는다?,,코인리더스,http://coinreaders.com
11115,2020.07.31,,"""로커스체인, 블록체인 '세계 표준' 가능""",,UPI뉴스,http://upinews.kr
11116,2020.07.31,,"中 공안, 사상 최대 암호화폐 스캠 '플러스토큰' 수사 마무리...주모자 포함 ...",,코인리더스,http://coinreaders.com


In [116]:
result.to_csv('비트코인_20200501_20200731.csv', encoding='utf-8-sig', index=False)